In [ ]:
import os
import requests
import csv
import linecache
import numpy as np
import pandas as pd

In [ ]:
#change these
filename = "20-71rx"
html_ = f"{filename}.html"
csv_ =  f"{filename}.csv"


In [ ]:
# Path
data_folder = "data"
 
# Join various path components
html_file = os.path.join(data_folder, html_)
print(html_file)

In [ ]:
#get meet name, usually on line 6, print to doublecheck
meet = linecache.getline(html_file, 6).strip()
print(meet)

In [ ]:
#Creating an output file in writing mode
csv_file = os.path.join(data_folder,csv_)

#csv header
header = ["meet", "event", "rank", "name", "age", "team", "seed", "finals"]
#assign 'delete' event for delition later
event = "delete"

with open(csv_file, 'w', encoding='UTF8') as f:
    #create a writer
    writer = csv.writer(f, delimiter = "|")
    # write the header
    writer.writerow(header)
    #iterate html
    with open(html_file, "r") as line:
        line_string = line.readlines()
        #this will assign event column
        for each_line in line_string:
            if each_line[0:2] == "<b":
                event = each_line.replace("<b>", "")\
                                 .replace("</b>", "")\
                                 .replace("&amp;", "&")\
                                 .replace("\n", " ")\
                                 .rstrip()
            else:
                #assign each column and strip right spaces
                rank = each_line[0:4].replace(" ", "").replace("=", "").rstrip()
                name = each_line[4:29].rstrip()
                age = each_line[28:33].replace(" ","").rstrip()
                team = each_line[33:50].replace(" ","").rstrip()
                seed = each_line[51:65].replace(" ","").rstrip()
                finals = each_line[61:80].replace(" ","").rstrip()
                #this will not write uncessary rows
                if rank != "" and event != 'delete'\
                   and seed != "" and finals != "":
                    writer.writerow([meet, event, rank, name, age, team, seed, finals])



In [ ]:
#read csv to data frame
df = pd.read_csv(csv_file, sep="|"  , engine="python")

In [ ]:
#filtering examples
filt = (df["name"] == "lname, fname") #&  (df["rank"] == "12")
#get data
df[filt]

In [ ]:
#this cell is cleaning up seed and finals time

#filter NT from final
filt_NT = (df["seed"] == "NT")
#update dq to 00.00
df["seed"][filt_NT] = "00.00"

#filter DQ from final
filt_DQ = (df["finals"] == "DQ")
#update dq to 00.00
df["finals"][filt_DQ] = "00.00"

In [ ]:
df["seed"].head()

In [ ]:
#craete function to convert time string to float
def f_totime(i):
    #find minute indicator
    minute_ind = i.find(':')
    if (minute_ind != -1):
        i = float(i[0:minute_ind]) * 60 + float(i[minute_ind:].replace(":", ""))
        return i
    else:
        return i

# #testing fuction
# i = "10:29.05"

# result = f_totime(i)
# print(result)

In [ ]:
#create new column from seed
df["seed_time"] = df.apply(lambda row: f_totime(row["seed"]),axis=1)
#create new column from final
df["finals_time"] = df.apply(lambda row: f_totime(row["finals"]),axis=1)

In [ ]:
#change data type for seed time and final time
df["finals_time"] = df["finals_time"].astype(np.float16)

df["seed_time"] = df["seed_time"].astype(np.float16)

#create delta column
df["delta"] = df["finals_time"] - df["seed_time"]


In [ ]:
#function converting delta gain/drop/same
def f_gain_drop(i):
    if i < 0:
        return "Gain"
    elif i > 0:
        return "Drop"
    else:
        return "Same"

In [ ]:
#create new column for drop/gain/same
df["gain_drop_same"] = df.apply(lambda row: f_gain_drop(row["delta"]),axis=1)

df.head()